In [10]:
import csv
import random
import datetime
import faker

# Initialize Faker library to generate fake data
fake = faker.Faker()

# Function to generate random email addresses
def generate_email():
    return fake.email()

# Function to extract domain from email address
def extract_domain(email):
    return email.split('@')[-1]

# Function to generate random timestamp in April 2023
def generate_timestamp():
    return fake.date_time_between_dates(datetime.datetime(2023, 4, 1), datetime.datetime(2023, 4, 30)).strftime("%Y-%m-%d %H:%M:%S")

# Function to generate random event type
def generate_event_type():
    return random.choice(["sent", "click", "open","unsub"])

# Generating data and writing to CSV file
with open("sfmc_csv/generated_data.csv", mode="w", newline="") as file:

    writer = csv.writer(file)
    writer.writerow(["id", "eventtype", "timestamp", "subscriberkey", "domain", "jobid"])

    for i in range(1000):  # Generating 1000 rows
        subscriberkey = generate_email()
        writer.writerow([i+1, generate_event_type(), generate_timestamp(), subscriberkey, extract_domain(subscriberkey), random.randint(100000, 999999)])

print("CSV file generated successfully.")


CSV file generated successfully.


In [2]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker to generate random data
fake = Faker()

def extract_domain(email):
    return email.split('@')[-1]

# Function to generate random timestamp in April
def random_timestamp():
    start = datetime(2024, 4, 1)
    end = datetime(2024, 4, 30)
    return fake.date_time_between(start, end)

def generate_unique_numbers(num_numbers):
    unique_numbers = []
    while len(unique_numbers) < num_numbers:
        new_number = random.randint(100000, 999999)
        unique_numbers.append(new_number)
    return unique_numbers

# Generate 20 random 6-digit numbers
random_numbers = generate_unique_numbers(20)
# Generate random email addresses and initialize DataFrame
num_subscribers = 1000
emails = [fake.email() for _ in range(num_subscribers)]
df = pd.DataFrame({'SubscriberKey': emails})

# Initialize event types and set for subscribers
event_types = ['sent', 'open', 'click', 'unsub']
events = {email: [] for email in emails}

# Generate events for each subscriber
for email in emails:
    unsub_added = False
    timestamp = None
    
    while not unsub_added:
        event_type = random.choice(event_types)
        
        if event_type == 'unsub':
            if not timestamp:  # Ensure unsub event has timestamp
                timestamp = random_timestamp()
            unsub_added = True
            events[email].append((event_type, timestamp))
        else:
            if not timestamp or event_type == 'sent':
                timestamp = random_timestamp()
            elif event_type == 'open':
                timestamp += timedelta(minutes=random.randint(1, 60))
            elif event_type == 'click':
                timestamp += timedelta(seconds=random.randint(1, 600))
            
            events[email].append((event_type, timestamp))

# Flatten events dictionary to list of dictionaries
event_records = []
for email, event_list in events.items():
    for event_type, timestamp in event_list:
        event_records.append({'SubscriberKey': email, 'Event': event_type, 'Domain': extract_domain(email), 'Timestamp': timestamp, 'JobID': random.choice(random_numbers)})

# Create DataFrame from event records and sort by timestamp
df_events = pd.DataFrame(event_records).sort_values(by=['SubscriberKey', 'Timestamp'])

# Write DataFrame to CSV
df_events.to_csv('sfmc_csv/subscriber_events.csv', index=False)
